# Generative AI Use Case: Summarize Dialogue

In [1]:
%pip install datasets
%pip install transformers
%pip install torch
%pip install torchdata


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\astaj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\astaj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\astaj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\astaj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GenerationConfig

C:\Users\astaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Found cached dataset csv (C:/Users/astaj/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c07c4cf4362c223c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 3/3 [00:00<00:00, 34.86it/s]


In [6]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print("Example {}".format(i+1))
    print(dash_line)
    print('Input Dialogue:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('baseline summary:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()
    

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
baseline summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example 2


In [5]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\astaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\astaj\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [8]:
sentence = "Hello, my dog is cute"

sentence_encoded = tokenizer(sentence, return_tensors="pt")

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True)

print("Encoded sentence: {}".format(sentence_encoded))
print("Decoded sentence: {}".format(sentence_decoded))


Encoded sentence: {'input_ids': tensor([[8774,    6,   82, 1782,   19, 5295,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Decoded sentence: Hello, my dog is cute


### Without any Prompt Engineering

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

---------------------------------

### Zero Shot Inference with an Instruction Prompt - Method-1

In [11]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Simmarize the following conversation.

    {dialogue}

    Summary:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to leave.

----------------------------------------------

### Zero Shot Inference with an Instruction Prompt - Method-2

In [12]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Dialogue:

    {dialogue}

    What was going on?
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT_2:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT_2:
Tom is late for the train.

----------------------------------------------

### One Shot Inference

In [52]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.

        prompt += f""""
Dialogue:

        {dialogue}

What was goin on?
        {summary}

        """
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f""""
Dilaogue:

        {dialogue}

What was going on?
        """
    return prompt

In [53]:
example_indices_full = [60]
example_index_to_summarize = 120

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

"
Dialogue:

        #Person1#: Hey, Frank. I heard you got a new job.
#Person2#: Yeah, Judy. I will be working for the Post Office. It's not a bad job.
#Person1#: Is it true that you have a heavy work schedule?
#Person2#: That's right. I am supposed to work at 5am everyday, and I only get 45 minutes for lunch.
#Person1#: So, why did you apply for such a demanding job?
#Person2#: Well, the government offers its employees excellent health insurance benefits.
#Person1#: Oh, I see. And can your family members utilize the health insurance, too?
#Person2#: Yeah, that's the best part. All of my children can get free medical care.
#Person1#: That's a great employment benefit!
#Person2#: Now you know why I wanted to work for the Post Office!

What was goin on?
        Frank got a new job and is telling Judy not only the heavy schedule but also the benefits of this job.

        "
Dilaogue:

        #Person1#: Hello, I bought the pendant in your shop, just before. 
#Person2#: Yes. Thank you ver

In [55]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
    )

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to change the broken pendant in #Person2#'s shop.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Dilaogue will replace the pendant with a new one.



### Few shot Inference

In [56]:
example_indices_full = [40,80,120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

"
Dialogue:

        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was goin on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

        "
Dialogue:

        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picn

In [57]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
    )

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Dilaogue suggests that Person1 should consider upgrading his system.



### 5 - Generative Configuration Parameters for Inference

In [61]:
#generation_config = GenerationConfig(max_new_tokens = 50)
#generation_config = GenerationConfig(max_new_tokens = 50)
#generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 0.1)
generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 2.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config,
    )[0],
    skip_special_tokens=True
    )

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}\n')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')


---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Delos will start to look ahead at using a home cinema device that includes multiple screens. Dilaogue's friend recommended painting softwares with it.

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

